In [139]:
import numpy as np 
import pandas as pd 
import string
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from textblob import Word
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [140]:
df = pd.read_csv("/Users/harshithar/Documents/Assignments/Spring_2023/BIA_686/project/NYC_Jobs.csv") 
df.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,533891,HRA/DEPT OF SOCIAL SERVICES,External,1,CONTRACT ANALYST,STAFF ANALYST,Competitive-1,12626,02,"Administration & Human Resources Policy, Resea...",...,**LOAN FORGIVENESS The federal government prov...,APPLICANTS MUST BE PERMANENT IN THE STAFF ANAL...,NaN,NaN,NaN,New York City residency is generally required ...,05/27/2022,NaN,11/07/2022,02/07/2023
1,554785,DEPARTMENT OF TRANSPORTATION,External,1,DESIGN ENGINEER,CIVIL ENGINEERING INTERN,Competitive-1,20202,00,"Engineering, Architecture, & Planning",...,As a current or prospective employee of the Ci...,RESUMES MUST BE SUBMITTED ELECTRONICALLY USING...,NaN,NaN,NaN,New York City Residency is not required for th...,10/12/2022,NaN,12/28/2022,02/07/2023
2,568481,DEPT OF DESIGN & CONSTRUCTION,Internal,1,Director,ADM PROC ANAL-NM FRM M1-3,Competitive-1,8297A,00,"Finance, Accounting, & Procurement",...,NaN,"For City Employees, please go to Employee Self...",35 Hours,"30-30 Thomson Ave, LIC, NY 11101",NaN,New York City residency is generally required ...,01/20/2023,21-MAR-2023,01/20/2023,02/07/2023
3,527833,NYC HOUSING AUTHORITY,External,1,ELECTRICIAN,ELECTRICIAN,Competitive-1,91717,00,Building Operations & Maintenance,...,"1.\tFor NYCHA employees, these positions are o...",Click the Apply Now button.,NaN,NaN,NaN,NYCHA has no residency requirements.,04/13/2022,NaN,05/12/2022,02/07/2023
4,541555,FIRE DEPARTMENT,External,1,"Director, Data Quality",CITY RESEARCH SCIENTIST,Non-Competitive-5,21744,4A,"Technology, Data & Innovation Policy, Research...",...,***LOAN FORGIVENESS: As a current or prospecti...,NON-CITY EMPLOYEES/EXTERNAL CANDIDATES PLEASE ...,NaN,NaN,NaN,New York City residency is generally required ...,08/10/2022,NaN,08/10/2022,02/07/2023


In [141]:
##to string
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].apply(lambda x: str(x))
## Lower case
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].apply(lambda x: " ".join(x.lower()for x in x.split()))
## remove special charecters
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].str.replace('[^a-zA-Z0-9]', ' ')
## digits
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].str.replace('\d+', '')

#remove stop words
stop = stopwords.words('english')
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## lemmatization
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

print("Preprocessed data: \n")
print(df['Minimum Qual Requirements'])

<ipython-input-141-b06c0e58a44f>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].str.replace('[^a-zA-Z0-9]', ' ')
<ipython-input-141-b06c0e58a44f>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].str.replace('\d+', '')


Preprocessed data: 

0       master degree accredited college university ac...
1       baccalaureate degree civil engineering accredi...
2       baccalaureate degree accredited college four y...
3       five year full time satisfactory experience el...
4       assignment level physical biological environme...
                              ...                        
5929    bachelor master degree landscape architecture ...
5930    graduation accredited college baccalaureate de...
5931    baccalaureate degree accredited college four y...
5932    baccalaureate degree accredited college three ...
5933    baccalaureate degree accredited college univer...
Name: Minimum Qual Requirements, Length: 5934, dtype: object


In [142]:
other_stop_words = ['junior', 'senior','etc','job','work','company','technique',
                    'candidate','skill','skills','language','menu','inc','new','plus','years',
                   'technology','organization','open','nyc','new', 'york','city','brooklyn','bronx','queens',
                    'account','manager','permanent','intern','citizen','department','may', 'office','service',
                   'described','note','agency','state','must','least','approved','satisfactory','duty', 
                    'however','including','difficulty','level','ii','nan']

In [143]:
df['Minimum Qual Requirements'] = df['Minimum Qual Requirements'].apply(lambda x: " ".join(x for x in x.split() if x not in other_stop_words))

In [144]:
df.dropna(subset = ['Minimum Qual Requirements'])

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,533891,HRA/DEPT OF SOCIAL SERVICES,External,1,CONTRACT ANALYST,STAFF ANALYST,Competitive-1,12626,02,"Administration & Human Resources Policy, Resea...",...,**LOAN FORGIVENESS The federal government prov...,APPLICANTS MUST BE PERMANENT IN THE STAFF ANAL...,NaN,NaN,NaN,New York City residency is generally required ...,05/27/2022,NaN,11/07/2022,02/07/2023
1,554785,DEPARTMENT OF TRANSPORTATION,External,1,DESIGN ENGINEER,CIVIL ENGINEERING INTERN,Competitive-1,20202,00,"Engineering, Architecture, & Planning",...,As a current or prospective employee of the Ci...,RESUMES MUST BE SUBMITTED ELECTRONICALLY USING...,NaN,NaN,NaN,New York City Residency is not required for th...,10/12/2022,NaN,12/28/2022,02/07/2023
2,568481,DEPT OF DESIGN & CONSTRUCTION,Internal,1,Director,ADM PROC ANAL-NM FRM M1-3,Competitive-1,8297A,00,"Finance, Accounting, & Procurement",...,NaN,"For City Employees, please go to Employee Self...",35 Hours,"30-30 Thomson Ave, LIC, NY 11101",NaN,New York City residency is generally required ...,01/20/2023,21-MAR-2023,01/20/2023,02/07/2023
3,527833,NYC HOUSING AUTHORITY,External,1,ELECTRICIAN,ELECTRICIAN,Competitive-1,91717,00,Building Operations & Maintenance,...,"1.\tFor NYCHA employees, these positions are o...",Click the Apply Now button.,NaN,NaN,NaN,NYCHA has no residency requirements.,04/13/2022,NaN,05/12/2022,02/07/2023
4,541555,FIRE DEPARTMENT,External,1,"Director, Data Quality",CITY RESEARCH SCIENTIST,Non-Competitive-5,21744,4A,"Technology, Data & Innovation Policy, Research...",...,***LOAN FORGIVENESS: As a current or prospecti...,NON-CITY EMPLOYEES/EXTERNAL CANDIDATES PLEASE ...,NaN,NaN,NaN,New York City residency is generally required ...,08/10/2022,NaN,08/10/2022,02/07/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5929,562393,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Junior Landscape Designer,ASSISTANT LANDSCAPE ARCHITECT,Competitive-1,21310,00,"Engineering, Architecture, & Planning",...,The City of New York is an inclusive equal opp...,To apply click Apply Now,NaN,NaN,NaN,New York City Residency is not required for th...,11/25/2022,NaN,11/28/2022,02/07/2023
5930,562919,DEPARTMENT FOR THE AGING,Internal,1,"Director, Health Insurance Information, Counse...",ADMINISTRATIVE PROGRAM OFFICE,Competitive-1,10084,M1,Constituent Services & Community Programs Heal...,...,NaN,Please be sure to submit a resume & cover lett...,NaN,NaN,NaN,New York City residency is generally required ...,01/19/2023,18-JUL-2023,01/20/2023,02/07/2023
5931,562782,DEPT OF HEALTH/MENTAL HYGIENE,External,1,"Solicitation Specifications/Writer, Office of ...",ADM PROC ANAL-NM FRM M1-3,Competitive-1,8297A,00,"Finance, Accounting, & Procurement",...,**IMPORTANT NOTES TO ALL CANDIDATES: Please n...,Apply online with a cover letter to https://a1...,NaN,NaN,NaN,New York City residency is generally required ...,12/09/2022,08-APR-2023,12/23/2022,02/07/2023
5932,541618,BRONX DISTRICT ATTORNEY,Internal,2,Timekeeper: Human Resources,PRINCIPAL ADMINISTRATIVE ASSOC,Competitive-1,10124,01,Administration & Human Resources Legal Affairs...,...,MUST BE A PRINCIPLE ADMINISTRATIVE ASSOCIATE L...,We appreciate your interest in a position with...,NaN,NaN,NaN,City Residency is not required for this position,07/21/2022,21-JUL-2023,02/02/2023,02/07/2023


In [145]:
df.isnull().sum()

Job ID                              0
Agency                              0
Posting Type                        0
# Of Positions                      0
Business Title                      0
Civil Service Title                 0
Title Classification                0
Title Code No                       0
Level                               0
Job Category                        2
Full-Time/Part-Time indicator     231
Career Level                        2
Salary Range From                   0
Salary Range To                     0
Salary Frequency                    0
Work Location                       0
Division/Work Unit                  0
Job Description                     0
Minimum Qual Requirements           0
Preferred Skills                  862
Additional Information           1204
To Apply                            1
Hours/Shift                      3834
Work Location 1                  3358
Recruitment Contact              5934
Residency Requirement               0
Posting Date

In [146]:
##to string
df['Job Description'] = df['Job Description'].apply(lambda x: str(x))
## Lower case
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join(x.lower()for x in x.split()))
## remove special charecters
df['Job Description'] = df['Job Description'].str.replace('[^a-zA-Z0-9]', ' ')
## digits
df['Job Description'] = df['Job Description'].str.replace('\d+', '')

#remove stop words
stop = stopwords.words('english')
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## lemmatization
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

print("Preprocessed data: \n")
print(df['Job Description'])

<ipython-input-146-64eafdec983e>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Job Description'] = df['Job Description'].str.replace('[^a-zA-Z0-9]', ' ')
<ipython-input-146-64eafdec983e>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Job Description'] = df['Job Description'].str.replace('\d+', '')


Preprocessed data: 

0       department social service ds comprised adminis...
1       division sidewalk inspection management sim re...
2       permanent administrative procurement analyst t...
3       direction electrician supervisor electrician a...
4       fire department city new york fdny largest fir...
                              ...                        
5929    nyc department environmental protection dep en...
5930    new york city home approximately million older...
5931    open candidate permanent administrative procur...
5932    bronx district attorney office seeking well qu...
5933    ds office audit service oas serf liaison exter...
Name: Job Description, Length: 5934, dtype: object


In [147]:
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join(x for x in x.split() if x not in other_stop_words))

In [148]:
##to string
df['Preferred Skills'] = df['Preferred Skills'].apply(lambda x: str(x))
## Lower case
df['Preferred Skills'] = df['Preferred Skills'].apply(lambda x: " ".join(x.lower()for x in x.split()))
## remove special charecters
df['Preferred Skills'] = df['Preferred Skills'].str.replace('[^a-zA-Z0-9]', ' ')
## digits
df['Preferred Skills'] = df['Preferred Skills'].str.replace('\d+', '')

#remove stop words
stop = stopwords.words('english')
df['Preferred Skills'] = df['Preferred Skills'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## lemmatization
df['Preferred Skills'] = df['Preferred Skills'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

print("Preprocessed data: \n")
print(df['Preferred Skills'])

<ipython-input-148-eaefcb6f5797>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Preferred Skills'] = df['Preferred Skills'].str.replace('[^a-zA-Z0-9]', ' ')
<ipython-input-148-eaefcb6f5797>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Preferred Skills'] = df['Preferred Skills'].str.replace('\d+', '')


Preprocessed data: 

0                                                     nan
1       experience related site grading roadway engine...
2       candidate excellent organizational interperson...
3                                                     nan
4       bachelor master degree field statistic analyti...
                              ...                        
5929                                                  nan
5930    selected candidate must posse least plus year ...
5931    excellent written oral interpersonal skill abi...
5932                                                  nan
5933    prior financial contract compliance operationa...
Name: Preferred Skills, Length: 5934, dtype: object


In [9]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [149]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.feature_extraction.text import CountVectorizer

## Converting text to features 
#vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer(ngram_range=(1,2))
#Tokenize and build vocabulary
X = vectorizer.fit_transform(df['Job Description'])
#X = vectorizer.fit_transform(df['Minimum Qual Requirements'])
y = df['Business Title']

# split data into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=109) 
print("train data shape: ",X_train.shape)
print("test data shape: ",X_test.shape)


## Fit model
#clf = MultinomialNB()
#clf = GaussianNB()
#clf.fit(X_train, y_train)
## Predict
#y_predicted = clf.predict(X_test)



# Fit model
xgb_clf = OneVsRestClassifier(XGBClassifier())
#clf = OneVsRestClassifier(LinearSVC())
xgb_clf.fit(X_train, y_train)
## Predict
y_predicted = xgb_clf.predict(X_test)

train data shape:  (4747, 217000)
test data shape:  (1187, 217000)


In [150]:
# check a few samples
print(y_predicted[0:3])
y_test[0:3]

['RECEPTIONIST' 'Deputy Director'
 'Public Health Inspector, Bureau of Food Safety and Community Sanitation (BFSCS)']


2814                                         RECEPTIONIST
5610                                      Deputy Director
3145    Public Health Inspector, Bureau of Food Safety...
Name: Business Title, dtype: object

In [151]:
#evaluate the predictions
print("Accuracy score is: ",accuracy_score(y_test, y_predicted))
print("Classes: (to help read Confusion Matrix)\n", clf.classes_)
print("Confusion Matrix: ")

print(confusion_matrix(y_test, y_predicted))
print("Classification Report: ")
print(classification_report(y_test, y_predicted))

Accuracy score is:  0.8096040438079192
Classes: (to help read Confusion Matrix)
 ['.NET Developer' '.Net Programmer/Analyst for the Division of HPD Tech'
 'ASSISTANT DEPUTY COMMISSIONER, NETWORK,TELEPHONY& CLIENT SER'
 'AUTOMATION INTERN' 'Account Security Administrator'
 'Analyst  Service Desk Operations' 'Application Associate  (Part Time)'
 'Application Developer' 'Application Security Administrator'
 'Applications Developer' 'Asset Management Specialist'
 'Associate Data Engineer' 'Automation Testing Engineer'
 'BUSINESS ANALYST' 'Blaze Architect' 'Business Analyst'
 'Business Analyst for the Division of HPD Tech'
 'Business Analyst â\x80\x93 Product Owner'
 'CARETAKER E - ELEVATOR DISPATCHER'
 'CERTIFIED IT ADMINISTRATOR (LAN/WAN)'
 'CERTIFIED IT ADMINISTRATOR (LAN/WAN), LEVEL 4'
 'CERTIFIED IT ADMINISTRATOR (WAN)'
 'CERTIFIED IT ADMINISTRATOR (WAN), Level 4'
 'CERTIFIED IT DEVELOPER (APPLICATIONS)' 'COLLEGE AIDE'
 'COMPUTER ASSOC (OPERATIONS)' 'COMPUTER ASSOCIATE (SOFTWARE)'
 'CO

/Users/harshithar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshithar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshithar/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harsh

In [136]:
len(clf.classes_)

194

In [231]:
import csv
with open('techskill.csv', 'r') as f:
    reader = csv.reader(f)
    technical_skills = list(reader)

In [232]:
technical_skills = technical_skills[0]

In [248]:
technical_skills

['ajenti',
 'django-suit',
 'django-xadmin',
 'flask-admin',
 'flower',
 'grappelli',
 'wooey',
 'algorithms',
 'pypattyrn',
 'python-patterns',
 'sortedcontainers',
 'libraries',
 'django-simple-captcha',
 'django-simple-spam-blocker',
 'django-compressor',
 'django-pipeline',
 'django-storages',
 'fanstatic',
 'fileconveyor',
 'flask-assets',
 'jinja-assets-compressor',
 'webassets',
 'audiolazy',
 'audioread',
 'beets',
 'dejavu',
 'django-elastic-transcoder',
 'eyed3',
 'id3reader',
 'm3u8',
 'mingus',
 'pyaudioanalysis',
 'pydub',
 'pyechonest',
 'talkbox',
 'timeside',
 'tinytag',
 'authomatic',
 'django-allauth',
 'django-oauth-toolkit',
 'flask-oauthlib',
 'oauthlib',
 'python-oauth2',
 'python-social-auth',
 'rauth',
 'sanction',
 'jose',
 'pyjwt',
 'python-jws',
 'python-jwt',
 'bitbake',
 'buildout',
 'platformio',
 'pybuilder',
 'scons',
 'django-cms',
 'djedi-cms',
 'feincms',
 'kotti',
 'mezzanine',
 'opps',
 'plone',
 'quokka',
 'wagtail',
 'widgy',
 'libraries',
 'beake

In [ ]:
feature_array = vectorizer.get_feature_names()
# number of overall model features
features_numbers = len(feature_array)
## max sorted features number
n_max = int(features_numbers * 0.1)
n_max
features_numbers

In [234]:
test_prob = pd.DataFrame(xgb_clf.predict_proba(X_test), columns=clf.classes_)

In [193]:
test_prob = test_prob.apply(lambda x: x.sort_values().values)

In [206]:
for i in range(0,len(clf.classes_)):
    #print("\n****" ,clf.classes_[i],"****\n")
    class_prob_indices_sorted = test_prob.iloc[:, i].argsort()[::-1]

In [210]:
test_prob['ZONING URBAN DESIGNER']

0       0.000660
1       0.000086
2       0.000110
3       0.000349
4       0.000267
          ...   
1182    0.000106
1183    0.000714
1184    0.000302
1185    0.000372
1186    0.000366
Name: ZONING URBAN DESIGNER, Length: 1187, dtype: float32

In [137]:
from textblob import TextBlob

#technical_skills = ['python', 'c','r', 'c++','java','hadoop','scala','flask','pandas','spark','scikit','learn',
#                    'numpy','php','sql','mysql','css','mongdb','nltk','fastai' , 'keras', 'pytorch','tensorflow',
#                   'linux','ruby','javascript','django','react','reactjs','ai','ui','tableau','excel']

technical_skills = ['python', 'c','r', 'c++','java','hadoop','scala','flask','pandas','spark','scikit-learn',
                    'numpy','php','sql','mysql','css','mongdb','nltk','fastai' , 'keras', 'pytorch','tensorflow',
                   'linux','ruby','javascript','django','react','reactjs','ai','ui','tableau', 'database','software',
                    'analysis', 'analytics','data','programming', 'cloud']


feature_array = vectorizer.get_feature_names()
# number of overall model features
features_numbers = len(feature_array)
## max sorted features number
n_max = int(features_numbers * 0.1)
#n_max = 100

##initialize output dataframe
output = pd.DataFrame()

for i in range(0,len(clf.classes_)):
    #print("\n****" ,clf.classes_[i],"****\n")
    class_prob_indices_sorted = clf.feature_log_prob_[i, :].argsort()[::-1]
    raw_skills = np.take(feature_array, class_prob_indices_sorted[:n_max])
    print("list of unprocessed skills :")
    print(raw_skills)
    
    ## Extract technical skills
    top_technical_skills= list(set(technical_skills).intersection(raw_skills)) #[:6]
    #s = set(technical_skills)
    #top_technical_skills = [x for x in raw_skills if x in s]
 

    # transform list to string
    txt = " ".join(raw_skills)
    blob = TextBlob(txt)
    #top 6 adjective
    top_adjectives = [w for (w, pos) in TextBlob(txt).pos_tags if pos.startswith("JJ")][:6]
    #print("Top 6 adjectives: ",top_adjectives)
    
    output = output.append({'job_title':clf.classes_[i],
                        'technical_skills':top_technical_skills,
                        'soft_skills':top_adjectives },
                       ignore_index=True)

/Users/harshithar/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [138]:
print(output)

                                             job_title  \
0                                       .NET Developer   
1    .Net Programmer/Analyst for the Division of HP...   
2    ASSISTANT DEPUTY COMMISSIONER, NETWORK,TELEPHO...   
3                                    AUTOMATION INTERN   
4                       Account Security Administrator   
..                                                 ...   
189                       WEB APPLICATION UI DEVELOPER   
190                                      WEB ARCHITECT   
191                            WEBSPHERE ADMINISTRATOR   
192               WTC Applications Development Manager   
193                                      Web Developer   

                                           soft_skills  \
0        [full, net, full, responsible, full, graphic]   
1    [tech, tenant, affordable, committed, technica...   
2    [direct, technical, complex, telephony, techni...   
3    [financial, financial, functional, write, fix,...   
4    [safe, r

In [72]:
output['technical_skills']

0       [database, data, analysis, programming, software]
1                                              [analysis]
2       [database, data, analysis, programming, software]
3                                                      []
4                                                      []
                              ...                        
2112                                                   []
2113                                                   []
2114                                                   []
2115                                                   []
2116                                                   []
Name: technical_skills, Length: 2117, dtype: object

In [74]:
print(output.T)

                                                               0     \
job_title                                            .NET Developer   
soft_skills       [major, full, equivalent, specialized, profess...   
technical_skills  [database, data, analysis, programming, software]   

                                                               1     \
job_title         .NET Developer, Bureau of Environmental Diseas...   
soft_skills       [accredited, professional, full, regional, nat...   
technical_skills                                         [analysis]   

                                                               2     \
job_title         .Net Programmer/Analyst for the Division of HP...   
soft_skills       [major, full, equivalent, specialized, profess...   
technical_skills  [database, data, analysis, programming, software]   

                                                               3     \
job_title                   2023-BLA-001-Undergraduate Legal Intern   
sof

In [73]:
output.to_csv('output_data.csv', index=False)

In [90]:
output.to_csv('output_jd_data.csv', index=False)